In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import matplotlib.pyplot as plt
import scipy
import dateutil.parser
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import requests
import json
from collections import defaultdict
import gzip
import csv
import random
from sklearn.metrics import jaccard_score 
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE
import math
import string
from scipy.spatial import distance

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt', encoding = "UTF-8"):
        yield eval(l)

In [3]:
# QUESTION 1

fp = os.path.join('data', "goodreads_reviews_comics_graphic.json")
with open(fp) as j:
    data = j.readlines()
    data = list(map(json.loads, data))

In [4]:
df = pd.DataFrame(data)[:20000]


In [5]:
df['review_text'] = df['review_text'].apply(lambda x: re.sub("[^\w|^ ]+","", x).lower())

In [6]:
train = df[:10000]
test = df[10000:]

In [7]:
# Unigram

countVec = CountVectorizer(ngram_range=(1,1), max_features = 1000, tokenizer=lambda txt: txt.split())
    #transform
countData = countVec.fit_transform(train['review_text'])

    #create dataframe
cvDataframeTrain=pd.DataFrame(countData.toarray(),columns=countVec.get_feature_names())

In [8]:
regr = linear_model.Ridge()
regr.fit(cvDataframeTrain, train.rating)

Ridge()

In [9]:
testData = countVec.transform(test['review_text'])
cvDataframeTest=pd.DataFrame(testData.toarray(),columns=countVec.get_feature_names())

In [10]:
# MSE
MSE = np.mean((regr.predict(cvDataframeTest) - test.rating)**2)
MSE

1.227067214304324

In [11]:
words = pd.DataFrame(list(zip(countVec.get_feature_names(), regr.coef_)), columns = ['words', 'coef'])

In [12]:
# Top 5 Positive unigrams
words.sort_values(by = ['coef'], ascending = False)[:5]

,words,coef
926,wait,0.428206
86,beautifully,0.411250
746,serie,0.371593
999,yourself,0.355561
9,5,0.340172


In [13]:
# Top 5 Negative unigrams
words.sort_values(by = ['coef'], ascending = True)[:5]

,words,coef
225,disappointing,-0.540662
113,boring,-0.531866
922,visuals,-0.411806
540,mangas,-0.368739
730,says,-0.345934


In [14]:
# Bigrams

countVec = CountVectorizer(ngram_range=(2,2), max_features = 1000, tokenizer=lambda txt: txt.split())
    #transform
countData = countVec.fit_transform(train['review_text'])

    #create dataframe
cvDataframeTrain=pd.DataFrame(countData.toarray(),columns=countVec.get_feature_names())

In [15]:
regr = linear_model.Ridge()
regr.fit(cvDataframeTrain, train.rating)

Ridge()

In [16]:
testData = countVec.transform(test['review_text'])
cvDataframeTest=pd.DataFrame(testData.toarray(),columns=countVec.get_feature_names())

In [17]:
# MSE
MSE = np.mean((regr.predict(cvDataframeTest) - test.rating)**2)
MSE

1.2853904634654927

In [18]:
words = pd.DataFrame(list(zip(countVec.get_feature_names(), regr.coef_)), columns = ['words', 'coef'])

In [19]:
# Top 5 Positive unigrams
words.sort_values(by = ['coef'], ascending = False)[:5]

,words,coef
266,forget to,0.772505
678,stay tuned,0.704170
190,cant wait,0.684432
2,5 stars,0.647770
593,or manga,0.572358


In [20]:
# Top 5 Negative unigrams
words.sort_values(by = ['coef'], ascending = True)[:5]

,words,coef
897,tuned for,-1.276263
852,thoughts below,-0.842866
806,the worst,-0.602966
998,your next,-0.599774
4,a bad,-0.549110


In [21]:
# Both Uni and Bigrams

countVec = CountVectorizer(ngram_range=(1,2), max_features = 1000, tokenizer=lambda txt: txt.split())
    #transform
countData = countVec.fit_transform(train['review_text'])

    #create dataframe
cvDataframeTrain=pd.DataFrame(countData.toarray(),columns=countVec.get_feature_names())

In [22]:
regr = linear_model.Ridge()
regr.fit(cvDataframeTrain, train.rating)

Ridge()

In [23]:
testData = countVec.transform(test['review_text'])
cvDataframeTest=pd.DataFrame(testData.toarray(),columns=countVec.get_feature_names())

In [24]:
# MSE

MSE = np.mean((regr.predict(cvDataframeTest) - test.rating)**2)
MSE

1.2199583191395396

In [25]:
words = pd.DataFrame(list(zip(countVec.get_feature_names(), regr.coef_)), columns = ['words', 'coef'])

In [26]:
# Top 5 Positive unigrams
words.sort_values(by = ['coef'], ascending = False)[:5]

,words,coef
672,reviews as,0.944096
984,xoxo,0.864363
894,tuned,0.781009
523,manga happy,0.672681
220,due to,0.666516


In [27]:
# Top 5 Negative unigrams
words.sort_values(by = ['coef'], ascending = True)[:5]

,words,coef
895,tuned for,-1.148310
87,as promos,-0.661355
534,miss your,-0.634169
670,review katies,-0.547478
985,xoxo katie,-0.540229


In [28]:
# QUESTION 2

In [29]:
punctuation = set(string.punctuation)
wordCount = defaultdict(int)
for d in data[:10000]:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

In [34]:
idf = defaultdict(int)
for d in data[:10000]:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in set(r.split()):
        idf[w] += 1
        
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]

In [35]:
tf = defaultdict(int)
r = ''.join([c for c in train['review_text'].iloc[0]])
for w in r.split():
    # Note = rather than +=, different versions of tf could be used instead
    tf[w] = 1
    
tfidf = dict(zip(words,[tf[w] * math.log2(len(data[:10000]) / idf[w]) for w in words]))
tfidfQuery = [tf[w] * math.log2(len(data[:10000]) / idf[w]) for w in words]

In [36]:
def Cosine(x1,x2):
    numer = 0
    norm1 = 0
    norm2 = 0
    for a1,a2 in zip(x1,x2):
        numer += a1*a2
        norm1 += a1**2
        norm2 += a2**2
    if norm1*norm2:
        return numer / math.sqrt(norm1*norm2)
    return 0

In [42]:
similarities = []
for rev2 in data[:10000]:
    tf = defaultdict(int)
    r = ''.join([c for c in rev2['review_text'].lower() if not c in punctuation])
    for w in r.split():
        # Note = rather than +=
        tf[w] = 1
    tfidf2 = [tf[w] * math.log2(len(data[:10000]) / idf[w]) for w in words]
    similarities.append((Cosine(tfidfQuery, tfidf2), rev2['review_text'], rev2['review_id']))

In [43]:
similarities.sort(reverse = True)
similarities[1]

(0.31303332977602777,
 'I am well aware that James Robinson isn\'t to every readers taste. That said, much of his style about which others complain are exactly the things I most love. He focuses on the basics of story telling -- character, setting, mood -- and never forgets that the excitement of action remains directly proportional to how much a reader cares about the characters. No where has he so demonstrated his gifts (and here fans of Golden Age may reasonably disagree) than on his magisterial Starman. "The Shade" gives readers the chance to revisit with some of Robinson\'s finest characters -- not the least of which being the Opal City, which while he wasn\'t the author to name, he was certainly the magician who brought it to life. \n As with Starman, Robinson demonstrates in "The Shade" the extraordinary depth of his knowledge of the Golden Age. I\'m pretty good on the topic, but even I found myself needing Google to locate Madam Fatal (a man who fought crime dressed as a woman!

In [99]:
# QUESTION 3

temp = df.groupby("user_id").size()
more_than_2_row = temp[temp>2]
df_more_than_2_row = more_than_2_row.reset_index()

In [100]:
ids = list(df_more_than_2_row.user_id.to_numpy())

In [101]:
dfInteractions = df[df['user_id'].isin(ids)]

In [102]:
dfInteractions['date_added'] = pd.to_datetime(dfInteractions['date_added'])

<ipython-input-102-cf7f8253e1db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInteractions['date_added'] = pd.to_datetime(dfInteractions['date_added'])


In [104]:
dfInteractions = dfInteractions.sort_values(by = ['user_id', 'date_added']).reset_index(drop = True)

In [105]:
testSet = []
train = dfInteractions
for n in range(1, dfInteractions.shape[0]):
    if dfInteractions.iloc[n-1]['user_id'] != dfInteractions.iloc[n]['user_id']:
        testSet.append(dfInteractions.iloc[n-1])
    else:
        train.drop(n-1)

In [106]:
test = pd.DataFrame(testSet)
train = train.reset_index(drop = True)

In [107]:
train

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,004d5e96c8a318aeb006af50f8cc949c,17571564,c0892c59cab3b0c2518dbcc61888186b,1,i am unsure if this book is supposed to be fun...,2014-01-04 11:28:18-08:00,Wed Jun 04 11:35:45 -0700 2014,Wed Jun 04 11:35:45 -0700 2014,Wed Jun 04 00:00:00 -0700 2014,1,0
1,004d5e96c8a318aeb006af50f8cc949c,15789361,713d2060267145692291a9acaab4ab23,3,i like the manga version of the story much mor...,2014-01-04 11:34:49-08:00,Tue Nov 11 20:50:25 -0800 2014,Tue Nov 11 00:00:00 -0800 2014,Fri Nov 07 00:00:00 -0800 2014,0,0
2,004d5e96c8a318aeb006af50f8cc949c,15704307,cec66320ca1704340bea3bd034fd642f,5,awesome beautiful graphics and a great story ...,2014-05-11 17:02:58-07:00,Thu Jan 14 17:25:20 -0800 2016,Wed Aug 13 16:35:34 -0700 2014,Wed Aug 13 00:00:00 -0700 2014,2,0
3,004d5e96c8a318aeb006af50f8cc949c,17131869,6466c273bb0e1962687c4f53b29b88ca,5,how can i use ordinary words to emphasize how ...,2014-11-11 20:47:49-08:00,Mon Feb 08 17:24:52 -0800 2016,Sun Dec 07 00:00:00 -0800 2014,Sun Dec 07 00:00:00 -0800 2014,1,0
4,004d5e96c8a318aeb006af50f8cc949c,21532236,198f2ef020a14fce7ab418c522ad68d0,4,i had very low expectations of this book as th...,2014-12-02 11:14:33-08:00,Mon May 25 10:52:40 -0700 2015,Mon May 25 10:52:40 -0700 2015,Thu May 21 00:00:00 -0700 2015,0,0
...,...,...,...,...,...,...,...,...,...,...,...
18922,ffec6cbb0db016bc371fc30b902c3166,29801,48227bad637a2b8b0ded2f93588e5577,4,better closer warmer,2010-05-20 06:08:53-07:00,Sun Jun 27 13:47:45 -0700 2010,Sat Jun 19 00:00:00 -0700 2010,Thu Jun 17 00:00:00 -0700 2010,0,0
18923,ffec6cbb0db016bc371fc30b902c3166,1481979,ab9cc25074f25a6ad49102b4f87d0cbe,1,probably the worst batman story i have ever re...,2010-06-01 08:38:08-07:00,Thu Jul 08 14:02:20 -0700 2010,Thu Jul 08 00:00:00 -0700 2010,Thu Jul 08 00:00:00 -0700 2010,0,0
18924,ffec6cbb0db016bc371fc30b902c3166,893506,0e53620e964d1827731e3d58a4272fe7,4,mignola wolverine awesomeness,2010-07-26 15:10:20-07:00,Tue Aug 17 19:33:52 -0700 2010,Wed Aug 04 00:00:00 -0700 2010,Tue Aug 03 00:00:00 -0700 2010,1,0
18925,ffec6cbb0db016bc371fc30b902c3166,1001525,56c9841aee50cd15f4226e8ead8854d0,3,my interest in this book was piqued solely due...,2011-02-11 13:28:47-08:00,Thu Apr 14 09:25:58 -0700 2011,Wed Apr 13 00:00:00 -0700 2011,Tue Apr 12 00:00:00 -0700 2011,0,0


In [2]:
pip install fastfm

  Using cached fastFM-0.2.10.tar.gz (1.6 MB)
  Running setup.py clean for fastfm
Failed to build fastfm
    Running setup.py install for fastfm: started
    Running setup.py install for fastfm: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'D:\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\laimy\\AppData\\Local\\Temp\\pip-install-grtop5mk\\fastfm_87fb30de866948c4b24add440b2f48cd\\setup.py'"'"'; __file__='"'"'C:\\Users\\laimy\\AppData\\Local\\Temp\\pip-install-grtop5mk\\fastfm_87fb30de866948c4b24add440b2f48cd\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\laimy\AppData\Local\Temp\pip-wheel-vxfddubn'
       cwd: C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\
  Complete output (133 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\fastFM
  copying fastFM\als.py -> build\lib.win-amd64-3.8\fastFM
  copying fastFM\base.py 

    C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\fastFM\../fastFM-core/externals/CXSparse/Include/cs.h(388): error C2059: syntax error: 'type'
    C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\fastFM\../fastFM-core/externals/CXSparse/Include/cs.h(388): error C2059: syntax error: ')'
    C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\fastFM\../fastFM-core/externals/CXSparse/Include/cs.h(389): error C2143: syntax error: missing ')' before '*'
    C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\fastFM\../fastFM-core/externals/CXSparse/Include/cs.h(389): error C2143: syntax error: missing '{' before '*'
    C:\Users\laimy\AppData\Local\Temp\pip-install-grtop5mk\fastfm_87fb30de866948c4b24add440b2f48cd\fastFM\../fastFM-core/externals/CXSparse/Include/cs.h(389): error C2059: syntax error: ')'
    C:\Us